In [108]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [119]:
DFRAME_PATH="/home/dm12h/authorship/temp/aggregated_df.csv"

In [ ]:
df = pd.read_csv(DFRAME_PATH)

In [4]:
df.head(10)

,author,text,text_no_punkt,lemmas,tags,tokens
0,Аверченко,Мы за пять лет. Материалы [к биографии]\nКак б...,Мы за пять лет Материалы к биографии Как будто...,мы пять год материал биография как кроваво-кра...,2_NPRO 4_NUMR 3_NOUN 9_NOUN 9_NOUN 3_CONJ 15_A...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Сидел он за большим письменным столом перед де...,Сидел он за большим письменным столом перед де...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...,Сидел он за большим письменным столом перед де...
2,Аверченко,"— Да уж, — качал головой сдержанный Ре-ми. — Н...",Да уж качал головой сдержанный Ре-ми Нехорошо ...,да качать голова сдержать ре-ми нехорошо нехор...,2_PRCL 5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_A...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,"Поверит ли кто-нибудь, что нами за эти пять ле...",Поверит ли кто-нибудь что нами за эти пять лет...,поверить кто-нибудь мы пять год совместно м.г ...,7_VERB 10_NPRO 4_NPRO 4_NUMR 3_NOUN 9_ADVB 3_N...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,"8\nАверченко А. Избранные рассказы. М., 1985. ...",8 Аверченко А Избранные рассказы М 1985 С 7 9 ...,8 аверченко а избранный рассказ м 1985 с 7 9 е...,1_None 9_NOUN 1_CONJ 9_ADJF 8_NOUN 1_NOUN 4_No...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."
5,Аверченко,"— Помилуйте, такой праздник… Нужно, обязательн...",Помилуйте такой праздник Нужно обязательно поц...,помиловать праздник нужно обязательно поцелова...,9_VERB 8_NOUN 5_PRED 11_ADVB 12_INFN 1_PREP 7_...,"— Помилуйте , такой праздник… Нужно , обязател..."
6,Аверченко,— Разбудить его в половине шестнадцатого.\n* *...,Разбудить его в половине шестнадцатого Закончи...,разбудить половина шестнадцатый закончить визи...,9_INFN 8_NOUN 13_ADJF 8_GRND 6_NOUN 7_NOUN 5_A...,— Разбудить его в половине шестнадцатого . * *...
7,Аверченко,"Он встал, оделся, побрился и вышел на улицу.\n...",Он встал оделся побрился и вышел на улицу На у...,он встать одеться побриться выйти улица на ули...,2_NPRO 5_VERB 6_VERB 8_VERB 5_VERB 5_NOUN 2_PR...,"Он встал , оделся , побрился и вышел на улицу ..."
8,Аверченко,Мозаика\nI\n— Я несчастный человек — вот что!\...,Мозаика I Я несчастный человек вот что Что за ...,мозаика i я несчастный человек что вздор никог...,7_NOUN 1_None 1_NPRO 10_ADJF 7_NOUN 3_CONJ 5_N...,Мозаика I — Я несчастный человек — вот что ! —...
9,Аверченко,— Как записываешь?\n— В записной книжке. Хочеш...,Как записываешь В записной книжке Хочешь У мен...,как записывать в записной книжка хотеть у мину...,3_CONJ 11_VERB 1_PREP 8_ADJF 6_NOUN 6_VERB 1_P...,— Как записываешь ? — В записной книжке . Хоче...


In [90]:
def get_document_vectorizer(frame, n_min=1, n=2, max_count=10000, column="lemmas"):
    texts_vector = frame[column]
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    vectorizer.fit(texts_vector)
    return vectorizer

def get_author_vectorizer(frame, n_min=1, n=2, max_count=10000, column="lemmas"):
    grouped_text = frame.groupby("author", as_index = False)[column].agg({column: ' '.join})
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    texts_vector = frame[column]
    vectorizer.fit(texts_vector)
    return vectorizer

In [52]:
def get_encoder(frame, column="author"):
    encoder = LabelEncoder()
    encoder.fit(frame[column])
    return encoder

### Попробуем в лоб:
возьмем лемматизированные отрезки текста, векторизируем их и попробуем предсказать

In [24]:
label_encoder = get_encoder(frame=df)

In [64]:
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [77]:
vectorizer = get_author_vectorizer(df_train, max_count=40000)
x_train = vectorizer.transform(df_train["lemmas"])
x_test = vectorizer.transform(df_test["lemmas"])

In [78]:
clf = LogisticRegression(class_weight="balanced",
                        max_iter=500)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [79]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.971622641509434

Если играться с параметрами, точность колеблется около 96-97%

### Попробуем на частях речи

In [ ]:
label_encoder = get_encoder(frame=df)
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [103]:
vectorizer = get_author_vectorizer(df_train, max_count=10000, column="tags", n_min=1, n=3)
x_train = vectorizer.transform(df_train["tags"])
x_test = vectorizer.transform(df_test["tags"])

In [104]:
clf = LogisticRegression(class_weight="balanced",
                         warm_start=False,
                         max_iter=500)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [105]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.6599245283018867

In [106]:
f1_score(clf.predict(x_train), y_train, average="micro")

0.8221618357487923

Через перебор n-gram и длины вектора точность на тесте не выше 0.66

### комбинируем

In [116]:
vectorizer1 = get_author_vectorizer(df_train, max_count=30000)
vectorizer2 = get_author_vectorizer(df_train, max_count=5000, column="tags", n_min=1, n=3)

x_train1 = vectorizer1.transform(df_train["lemmas"])
x_train2 = vectorizer2.transform(df_train["tags"])

x_test1 = vectorizer1.transform(df_test["lemmas"])
x_test2 = vectorizer2.transform(df_test["tags"])

x_train = sp.sparse.hstack([x_train1, x_train2])
x_test = sp.sparse.hstack([x_test1, x_test2])

In [117]:
clf = LogisticRegression(class_weight="balanced",
                         warm_start=False,
                         max_iter=500)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [118]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.9559245283018868

комбинация ничего особо не дает. Не похоже на переобучение, при снижении количества параметров мы ничего особо не выигрываем